In [6]:
import pickle
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
#load data to save to database
with open('parsed_match_data.pkl', 'rb') as f:
    parsed_match_data = pickle.load(f)

print(len(parsed_match_data[0]))

9655


In [29]:
def create_match_data_table():
    #create connection to db
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('DB_PORT')
    )

    cursor = conn.cursor()
    
    try:
        cursor.execute("""
            CREATE TABLE match_data (
                id SERIAL PRIMARY KEY,
                region VARCHAR(10),
                game_mode VARCHAR(50),
                patch VARCHAR(50),
                blue_team TEXT[],
                red_team TEXT[],
                winning_team INTEGER
            )
        """)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    conn.commit()

    cursor.close()
    conn.close()

create_match_data_table()

In [30]:
def insert_match_data(parsed_match_data):
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    cursor = conn.cursor()

    insert_sql = """
        INSERT INTO match_data (region, game_mode, patch, blue_team, red_team, winning_team)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    try:
        cursor.executemany(insert_sql,parsed_match_data)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    conn.close()
    cursor.close()
insert_match_data(parsed_match_data[0])

In [34]:
#test database to make sure data is inside
def get_match_data():
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    cursor = conn.cursor()

    query_sql = "SELECT * FROM match_data LIMIT 1"

    try:
        cursor.execute(query_sql)
        row = cursor.fetchone()

        if row is not None:
            print(row)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

get_match_data()

(1, 'NA1', 'ARAM', '14.13.596.7996', ['Rell', 'Velkoz', 'Samira', 'TwistedFate', 'Bard'], ['Tristana', 'Lux', 'Neeko', 'Lulu', 'Belveth'], 0)
